# A simple tax calculator

We all pay taxes, right?  In this example, we're going to pretend we live in a country with simple tax rules.


```
tax_owed = tax_rate * income
```

_Note: This is a simple example and is not indicative of my beliefs about how tax systems should work!!!_

In [1]:
import paramtools

class Taxes(paramtools.Parameters):
    defaults = {
        "tax_rate": {
            "title": "Tax Rate",
            "description": "Percentage of income that goes to the man.",
            "type": "float",
            "value": 0.1,
            "validators": {
                "range": {
                    "min": 0, "max": 1
                }
            }
        }
    }
    
    def run(self, income):
        return self.tax_rate[0]["value"] * income

In [2]:
taxes = Taxes()

Taxes owed for someone making $50,000/year

In [3]:
taxes.run(50000)

5000.0

**Earlier we set up some validation rules--what happens when we break them?**

In [4]:
taxes.adjust({"tax_rate": -0.1})

ValidationError: {
    "errors": {
        "tax_rate": [
            "tax_rate -0.1 < min 0 "
        ]
    }
}

## Tax rates change over time. Suppose the government wants to raise taxes in 2021 to 15% of income.

Let's add a year label to the Taxes class so we can update the tax rules in 2021.

In [5]:
class Taxes(paramtools.Parameters):
    defaults = {
        "schema": {
            "labels": {
                "year": {
                    "type": "int", 
                    "validators": {"range": {"min": 2020, "max": 2030}}
                }
            }
        },
        "tax_rate": {
            "title": "Tax Rate",
            "description": "Percent of income that goes to the man.",
            "type": "float",
            "value": [
                {"year": 2020, "value": 0.1},
            ],
            "validators": {
                "range": {
                    "min": 0, "max": 1
                }
            }
        }
    }
    
    def run(self, income, year):
        tax_rate = self.sel["tax_rate"]["year"] == year
        return tax_rate[0]["value"] * income

In [6]:
taxes = Taxes()

In [7]:
taxes.run(50000, year=2020)

5000.0

Update the tax rate in 2021 to be 0.15:

In [8]:
taxes.adjust({
    "tax_rate": [{"year": 2021, "value": 0.15}]
})

taxes.sel["tax_rate"]

Values([
  {'year': 2020, 'value': 0.1},
  {'year': 2021, 'value': 0.15},
])

In [9]:
taxes.run(50000, year=2021)

7500.0

## Calculate taxes for multiple years

The list of dicts starts to feel a bit cumbersome:

```python
    def run_all(self, income):
        return {value["year"]: self.run(income, year) for year in range(2020, 2030 + 1)}
 ```
 
 Enter: NumPy arrays.

In [10]:
class MultiYearTaxes(paramtools.Parameters):
    defaults = {
        "schema": {
            "labels": {
                "year": {
                    "type": "int", 
                    "validators": {"range": {"min": 2020, "max": 2030}}
                }
            },
            "operators": {
                "array_first": True
            }
        },
        "tax_rate": {
            "title": "Tax Rate",
            "description": "Percent of income that goes to the man.",
            "type": "float",
            "value": [
                {"year": 2020, "value": 0.1},
                {"year": 2021, "value": 0.1},
                {"year": 2022, "value": 0.1},
                {"year": 2023, "value": 0.1},
                {"year": 2024, "value": 0.1},
                {"year": 2025, "value": 0.1},
                {"year": 2026, "value": 0.1},
                {"year": 2027, "value": 0.1},
                {"year": 2028, "value": 0.1},
                {"year": 2029, "value": 0.1},
                {"year": 2030, "value": 0.1},
            ],
            "validators": {
                "range": {
                    "min": 0, "max": 1
                }
            }
        }
    }


    def run_all(self, income):
        return self.tax_rate * income


In [11]:
taxes = MultiYearTaxes()

taxes.tax_rate

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [12]:
taxes.run_all(50000)

array([5000., 5000., 5000., 5000., 5000., 5000., 5000., 5000., 5000.,
       5000., 5000.])

**But, what happens if we want to increase taxes in 2021 and going forward?**

In [13]:
taxes.adjust({
    "tax_rate": [{"year": 2021, "value": 0.15}]
})

taxes.sel["tax_rate"]

Values([
  {'year': 2020, 'value': 0.1},
  {'year': 2021, 'value': 0.15},
  {'year': 2022, 'value': 0.1},
  {'year': 2023, 'value': 0.1},
  {'year': 2024, 'value': 0.1},
  {'year': 2025, 'value': 0.1},
  {'year': 2026, 'value': 0.1},
  {'year': 2027, 'value': 0.1},
  {'year': 2028, 'value': 0.1},
  {'year': 2029, 'value': 0.1},
  {'year': 2030, 'value': 0.1},
])

In [14]:
taxes.adjust({
    "tax_rate": [
        {"year": year, "value": 0.15} for year in range(2021, 2030 + 1)
    ]
})

taxes.sel["tax_rate"]

Values([
  {'year': 2020, 'value': 0.1},
  {'year': 2021, 'value': 0.15},
  {'year': 2022, 'value': 0.15},
  {'year': 2023, 'value': 0.15},
  {'year': 2024, 'value': 0.15},
  {'year': 2025, 'value': 0.15},
  {'year': 2026, 'value': 0.15},
  {'year': 2027, 'value': 0.15},
  {'year': 2028, 'value': 0.15},
  {'year': 2029, 'value': 0.15},
  {'year': 2030, 'value': 0.15},
])

## Use `label_to_extend` to extend parameter values for multiple years

In [16]:
class MultiYearTaxes(paramtools.Parameters):
    defaults = {
        "schema": {
            "labels": {
                "year": {
                    "type": "int", 
                    "validators": {"range": {"min": 2020, "max": 2030}}
                }
            },
            "operators": {
                # Extend tax rate using the year label.
                "label_to_extend": "year",
                "array_first": True
            }
        },
        "tax_rate": {
            "title": "Tax Rate",
            "description": "Percent of income that goes to the man.",
            "type": "float",
            "value": [
                {"year": 2020, "value": 0.1},
            ],
            "validators": {
                "range": {
                    "min": 0, "max": 1
                }
            }
        }
    }
    
    def run_all(self, income):
        return self.tax_rate * income


In [17]:
taxes = MultiYearTaxes()
taxes.adjust({
    "tax_rate": [{"year": 2021, "value": 0.15}]
})

taxes.sel["tax_rate"]

Values([
  {'year': 2020, 'value': 0.1},
  {'year': 2021, 'value': 0.15},
  {'year': 2022, 'value': 0.15, '_auto': True},
  {'year': 2023, 'value': 0.15, '_auto': True},
  {'year': 2024, 'value': 0.15, '_auto': True},
  {'year': 2025, 'value': 0.15, '_auto': True},
  {'year': 2026, 'value': 0.15, '_auto': True},
  {'year': 2027, 'value': 0.15, '_auto': True},
  {'year': 2028, 'value': 0.15, '_auto': True},
  {'year': 2029, 'value': 0.15, '_auto': True},
  {'year': 2030, 'value': 0.15, '_auto': True},
])

In [18]:
taxes.run_all(50000)

array([5000., 7500., 7500., 7500., 7500., 7500., 7500., 7500., 7500.,
       7500., 7500.])

# Now this is starting to look like a real tax calculator.

One last concept is the idea of indexing values to an inflation rate.

To do this, we are going to add a parameter `deduction` that is indexed to inflation. The new tax equation is:

```
tax_owed = tax_rate * (income - deduction)
```

In [19]:
import numpy as np

class IndexedTaxes(paramtools.Parameters):
    defaults = {
        "schema": {
            "labels": {
                "year": {
                    "type": "int", 
                    "validators": {"range": {"min": 2020, "max": 2030}}
                }
            },
            "operators": {
                "label_to_extend": "year",
                "array_first": True,
                "uses_extend_func": True,
            }
        },
        "tax_rate": {
            "title": "Tax Rate",
            "description": "Percent of income that goes to the man.",
            "type": "float",
            "value": [
                {"year": 2020, "value": 0.1},
            ],
            "validators": {
                "range": {
                    "min": 0, "max": 1
                }
            }
        },
        "deduction": {
            "title": "Deduction",
            "description": "Amount you deduct from your income before taxes.",
            "type": "float",
            "indexed": True,
            "value": [
                {"year": 2020, "value": 10000}
            ],
            "validators": {
                "range": {"min": 0}
            }
        }
    }
    index_rates = {
        2020: 0.0073,
        2021: 0.0095,
        2022: 0.0158,
        2023: 0.0193,
        2024: 0.02,
        2025: 0.0202,
        2026: 0.0197,
        2027: 0.0199,
        2028: 0.0197,
        2029: 0.0199,
        2030: 0.0197
    }
    
    def calculate_adjusted_income(self, income):
        """Adjust income for inflation."""
        adjusted_income = [income]
        for i, year in enumerate(range(2021, 2031)):
            adjusted_income.append(
                adjusted_income[year - 2021] * (1 + self.index_rates[year])
            )
        return np.array(adjusted_income)
    
    
    def run_all(self, income):
        adjusted_income = self.calculate_adjusted_income(income)
        return self.tax_rate * (adjusted_income - self.deduction)

In [20]:
taxes = IndexedTaxes()

In [21]:
taxes.deduction

array([10000.  , 10073.  , 10168.69, 10329.36, 10528.72, 10739.29,
       10956.22, 11172.06, 11394.38, 11618.85, 11850.07])

In [22]:
taxes.run_all(50000)

array([4000.        , 4040.2       , 4110.3815    , 4193.27043465,
       4277.85856334, 4364.48232072, 4449.92602374, 4538.69842941,
       4627.88774667, 4720.21052903, 4812.96581095])